Exploration Fichiers CSV


In [22]:
import pandas as pd
import gdown


In [4]:
# Liens vers les fichiers de données IMDb
# Ces fichiers sont disponibles en téléchargement sur le site IMDb

link_title_basics = "https://datasets.imdbws.com/title.basics.tsv.gz"
link_title_akas = "https://datasets.imdbws.com/title.akas.tsv.gz"
link_title_crew = "https://datasets.imdbws.com/title.crew.tsv.gz"
link_title_episode = "https://datasets.imdbws.com/title.episode.tsv.gz"
link_title_principals = "https://datasets.imdbws.com/title.principals.tsv.gz"
link_title_ratings = "https://datasets.imdbws.com/title.ratings.tsv.gz"
link_name_basics = "https://datasets.imdbws.com/name.basics.tsv.gz"
link_tmdb = "https://drive.google.com/file/d/1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v/view"

In [25]:
# Fichier TMDB
file_id = "1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v"

output = "tmdb_movies.csv"

gdown.download(id=file_id,
               output=output,
               quiet=False,      # affiche la barre de progression
               fuzzy=True)  

Downloading...
From (original): https://drive.google.com/uc?id=1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v
From (redirected): https://drive.google.com/uc?id=1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v&confirm=t&uuid=bba747d7-01cb-4f46-835a-cdab91348c22
To: c:\Users\User\Documents\Projet2\WildCodeSchool-Projet2\tmdb_movies.csv
100%|██████████| 157M/157M [00:05<00:00, 29.7MB/s] 


'tmdb_movies.csv'

In [11]:
df_title_basics = pd.read_csv(link_title_basics, sep="\t",chunksize=1000, low_memory=False)

df_title_basics.get_chunk(100)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short
...,...,...,...,...,...,...,...,...,...
95,tt0000097,short,"Place de l'Opéra, First View","Place de l'Opéra, 1er aspect",0,1896,\N,\N,"Documentary,Short"
96,tt0000098,short,"Place de l'Opéra, Second View","Place de l'Opéra, 2e aspect",0,1896,\N,\N,"Documentary,Short"
97,tt0000099,short,Place de la Bastille,Place de la Bastille,0,1896,\N,\N,"Documentary,Short"
98,tt0000100,short,Place de la Concorde,Place de la Concorde,0,1896,\N,\N,"Documentary,Short"


In [10]:
df_title_akas = pd.read_csv(link_title_akas, sep="\t", chunksize=1000, low_memory=False)
df_title_akas.get_chunk(100)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,\N,\N,original,\N,1
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita,US,\N,imdbDisplay,\N,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
4,tt0000001,5,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...
95,tt0000010,24,Workers Leaving the Lumière Factory,US,\N,imdbDisplay,\N,0
96,tt0000010,25,Wyjście robotników z fabryki,PL,\N,imdbDisplay,\N,0
97,tt0000010,26,Вихід робітників із фабрики,UA,\N,imdbDisplay,\N,0
98,tt0000010,27,Выход рабочих с фабрики «Люмьер»,RU,\N,imdbDisplay,\N,0


In [14]:
df_title_crew = pd.read_csv(link_title_crew, sep="\t", low_memory=False)
df_title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,nm0721526
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
11619553,tt9916848,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
11619554,tt9916850,nm1485677,"nm9187127,nm1485677,nm9826385,nm1628284"
11619555,tt9916852,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
11619556,tt9916856,nm10538645,nm6951431


In [16]:
df_title_episode = pd.read_csv(link_title_episode, chunksize=1000, sep="\t", low_memory=False)
df_title_episode.get_chunk(100)

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0031458,tt32857063,\N,\N
1,tt0041951,tt0041038,1,9
2,tt0042816,tt0989125,1,17
3,tt0042889,tt0989125,\N,\N
4,tt0043426,tt0040051,3,42
...,...,...,...,...
95,tt0055130,tt0190181,2,5
96,tt0055131,tt0161126,4,27
97,tt0055132,tt0190181,2,2
98,tt0055203,tt0161126,4,47


In [17]:
df_title_principals = pd.read_csv(link_title_principals, sep="\t", chunksize=1000, low_memory=False)
df_title_principals.get_chunk(100)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0005690,producer,producer,\N
3,tt0000001,4,nm0374658,cinematographer,director of photography,\N
4,tt0000002,1,nm0721526,director,\N,\N
...,...,...,...,...,...,...
95,tt0000024,3,nm0010291,director,\N,\N
96,tt0000024,4,nm0010291,producer,producer,\N
97,tt0000024,5,nm0666972,producer,producer,\N
98,tt0000024,6,nm0010291,cinematographer,\N,\N


In [18]:
df_title_ratings = pd.read_csv(link_title_ratings, sep="\t", chunksize=1000, low_memory=False)
df_title_ratings.get_chunk(100)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2153
1,tt0000002,5.5,292
2,tt0000003,6.5,2189
3,tt0000004,5.3,187
4,tt0000005,6.2,2936
...,...,...,...
95,tt0000097,4.9,36
96,tt0000098,5.3,39
97,tt0000099,4.7,35
98,tt0000100,4.7,65


In [19]:
df_name_basics = pd.read_csv(link_name_basics, sep="\t", chunksize=1000, low_memory=False)
df_name_basics.get_chunk(100)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"
...,...,...,...,...,...,...
95,nm0000096,Gillian Anderson,1968,\N,"actress,producer,director","tt0106179,tt2294189,tt0455590,tt0442632"
96,nm0000097,Pamela Anderson,1967,\N,"actress,producer,director","tt31193791,tt0096542,tt18376122,tt0101120"
97,nm0000098,Jennifer Aniston,1969,\N,"actress,producer,director","tt0108778,tt3442006,tt1723121,tt0279113"
98,nm0000099,Patricia Arquette,1968,\N,"actress,producer,director","tt0108399,tt1065073,tt0412175,tt0145531"


In [24]:
df_tmdb = pd.read_csv(output, low_memory=False)

df_tmdb.head()

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
0,False,/dvQj1GBZAZirz1skEEZyWH2ZqQP.jpg,0,['Comedy'],NaN,3924,tt0029927,en,Blondie,Blondie and Dagwood are about to celebrate the...,...,70,['en'],Released,The favorite comic strip of millions at last o...,Blondie,False,7.214,7,['Columbia Pictures'],['US']
1,False,NaN,0,['Adventure'],NaN,6124,tt0011436,de,Der Mann ohne Namen,NaN,...,420,[],Released,NaN,"Peter Voss, Thief of Millions",False,0.000,0,[],[]
2,False,/uJlc4aNPF3Y8yAqahJTKBwgwPVW.jpg,0,"['Drama', 'Romance']",NaN,8773,tt0055747,fr,L'Amour à vingt ans,Love at Twenty unites five directors from five...,...,110,"['it', 'ja', 'pl', 'fr', 'de']",Released,The Intimate Secrets of Young Lovers,Love at Twenty,False,6.700,41,"['Ulysse Productions', 'Unitec Films', 'Cinese...","['', 'NZ', 'IT', 'JP', 'DE', 'PL', '']"
3,False,/hQ4pYsIbP22TMXOUdSfC2mjWrO0.jpg,0,"['Drama', 'Comedy', 'Crime']",NaN,2,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,...,73,['fi'],Released,NaN,Ariel,False,7.046,248,['Villealfa Filmproductions'],['FI']
4,False,/l94l89eMmFKh7na2a1u5q67VgNx.jpg,0,"['Drama', 'Comedy', 'Romance']",NaN,3,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",...,76,['en'],Released,NaN,Shadows in Paradise,False,7.182,269,['Villealfa Filmproductions'],['FI']
